In [ ]:
import glob
import pandas as pd
import numpy as np
import os
import csv
import sys
from pathlib import Path
import random
import datetime

from functools import reduce

import time 
from IPython.display import clear_output


In [ ]:
#Read CSV file containing the agents from SPENSER
df_persons_SPENSER__dir = r'' # use your path
df_persons_SPENSER__file = os.path.join(df_persons_SPENSER__dir, "")
df_persons_SPENSER = pd.read_csv(df_persons_SPENSER__file, index_col=None, header=0)

In [ ]:
df_persons_SPENSER_location_origin = df_persons_SPENSER[['PID_AreaMSOA', 'Area_MSOA', 'Area_OA', 'HID_AreaOA','Age']]

In [ ]:
df_persons_SPENSER_location_origin.head()

In [ ]:
len(df_persons_SPENSER_location_origin)

In [ ]:
header_students_matched =["PID_AreaMSOA", "IndividualID", "DayID", "School_PSU", "X", "Y", "School_OA", "School_MSOA"]

In [ ]:
#Read CSV file containing the agents from SPENSER
school_destination_dataframe_capacity__dir = r'' # use your path
school_destination_dataframe_capacity__file = os.path.join(school_destination_dataframe_capacity__dir, "")
df_school_destination_dataframe_capacity = pd.read_csv(school_destination_dataframe_capacity__file, index_col=None, header=0, names= header_students_matched)

In [ ]:
df_school_destination_dataframe_capacity.head()

In [ ]:
len(df_school_destination_dataframe_capacity)

In [ ]:
spenser_children_nts_ind_id_list = df_school_destination_dataframe_capacity['IndividualID'].unique().tolist()
spenser_children_nts_day_id_list = df_school_destination_dataframe_capacity['DayID'].unique().tolist()

In [ ]:
# Merge both df to have the oa level
df_school_destination_dataframe_capacity = pd.merge(df_school_destination_dataframe_capacity, df_persons_SPENSER_location_origin,  how='left', left_on=['PID_AreaMSOA'], right_on = ['PID_AreaMSOA'])

In [ ]:
df_school_destination_dataframe_capacity.head()

In [ ]:
len(df_school_destination_dataframe_capacity)

In [ ]:
#Read CSV file containing the matches between NTS and SPENSER:
NE_Facilities_dir = r'' # use your path
NE_Facilities_file = os.path.join(NE_Facilities_dir, "")
df_NE_Facilities = pd.read_csv(NE_Facilities_file, index_col=None, header=0)

In [ ]:
## Remove duplicate rows based on IndividualID and DayID (if they have more than one trip, keep only the first (the longest))
df_NE_Facilities = df_NE_Facilities.drop_duplicates(subset=['id'], keep="first")


In [ ]:
## update the names of 'leisure' to 'leisure_act'
### This is required because if searching for 'leisure', then 'leisure_sport' will be considered as well

# First, change leisure_sport to something else (i.e., 'sport_activity')
df_NE_Facilities['activities_2'] = df_NE_Facilities['activities'].str.replace('leisure_sport','sport_activity')

# Second, change 'leisure' to 'leisure_act'
df_NE_Facilities['activities_3'] = df_NE_Facilities['activities_2'].str.replace('leisure','leisure_act')

# Third, change again 'sport_activity' back to 'leisure_sport'
df_NE_Facilities['activities_4'] = df_NE_Facilities['activities_3'].str.replace('sport_activity','leisure_sport')

# Remove columns:
df_NE_Facilities.drop(['activities','activities_2', 'activities_3'], axis=1, inplace=True)

#Rename column
df_NE_Facilities.rename(columns={'activities_4': 'activities'}, inplace=True)

In [ ]:
df_NE_Facilities

In [ ]:
#Read CSV file containing the matches between NTS and SPENSER:
NTS_trips_fir = r'' # use your path
NTS_trips_file = os.path.join(NTS_trips_fir, "")
df_NTS_trips = pd.read_csv(NTS_trips_file, index_col=None, header=0)

In [ ]:
len(df_NTS_trips)

In [ ]:
# Select only those trips related to the emmployed people
df_NTS_trips = df_NTS_trips.loc[(df_NTS_trips['IndividualID'].isin(spenser_children_nts_ind_id_list) ) & (df_NTS_trips['DayID'].isin(spenser_children_nts_day_id_list) )]

In [ ]:
len(df_NTS_trips)

In [ ]:
df_NTS_trips.sort_values('TripID')

In [ ]:
###############################

In [ ]:
#Read CSV file containing the distances between msoa centroids (meters and miles) 
MSOA_lelvel_centroid_distances_dir = r'' # use your path
MSOA_lelvel_centroid_distances_file = os.path.join(MSOA_lelvel_centroid_distances_dir, "")
df_MSOA_lelvel_centroid_distances = pd.read_csv(MSOA_lelvel_centroid_distances_file, index_col=None, header=0)

In [ ]:
df_MSOA_lelvel_centroid_distances = df_MSOA_lelvel_centroid_distances[['msoa_origin', 'msoa_destination', 'distance_miles']]

In [ ]:
# If distance is 0, then update the value to 0.001
df_MSOA_lelvel_centroid_distances.loc[df_MSOA_lelvel_centroid_distances['distance_miles'] == 0, 'distance_miles'] = 0.001

In [ ]:
#Read CSV file containing the distances between msoa centroids (meters and miles) 
msoa_facility_types_dir = r'' # use your path
msoa_facility_types_file = os.path.join(msoa_facility_types_dir, "")
df_msoa_facility_types = pd.read_csv(msoa_facility_types_file, index_col=None, header=0)

In [ ]:
# Merge both df to have the oa level
df_MSOA_level_distance_and_facilities = pd.merge(df_MSOA_lelvel_centroid_distances, df_msoa_facility_types,  how='left', left_on=['msoa_destination'], right_on = ['msoa_name'])

In [ ]:
# Remove column oa_name
df_MSOA_level_distance_and_facilities.drop('msoa_name', axis=1, inplace=True)

In [ ]:
# If distance is 0, then update the value to 0.001
df_MSOA_level_distance_and_facilities.loc[df_MSOA_level_distance_and_facilities['distance_miles'] == 0, 'distance_miles'] = 0.001

In [ ]:
###############################

In [ ]:
#Read CSV file containing the distances between msoa centroids (meters and miles) 
OA_lelvel_centroid_distances_dir = r'' # use your path
OA_lelvel_centroid_distances_file = os.path.join(OA_lelvel_centroid_distances_dir, "")
df_OA_lelvel_centroid_distances = pd.read_csv(OA_lelvel_centroid_distances_file, index_col=None, header=0)

In [ ]:
df_OA_lelvel_centroid_distances.rename(columns={'oa11cd_x': 'oa_origin', 'oa11cd_y': 'oa_destination', 'distance': 'distance_meters'}, inplace=True)

In [ ]:
df_OA_lelvel_centroid_distances['distance_miles'] = df_OA_lelvel_centroid_distances['distance_meters']/1609

In [ ]:
# create a list of the buildings with the following attributes only: 'PID_AreaMSOA', 'Area_MSOA', 'Area_OA' 
df_OA_lelvel_centroid_distances = df_OA_lelvel_centroid_distances[['oa_origin', 'oa_destination', 'distance_miles']]

In [ ]:
df_OA_lelvel_centroid_distances.dtypes

In [ ]:
# If distance is 0, then update the value to 0.001
df_OA_lelvel_centroid_distances.loc[df_OA_lelvel_centroid_distances['distance_miles'] == 0, 'distance_miles'] = 0.001

In [ ]:
####################################

In [ ]:
#Read CSV file containing the distances between msoa centroids (meters and miles) 
oa_facility_types_dir = r'' # use your path
oa_facility_types_file = os.path.join(oa_facility_types_dir, "")
df_oa_facility_types = pd.read_csv(oa_facility_types_file, index_col=None, header=0)

In [ ]:
# Merge both df to have all info together
df_OA_level_distance_and_facilities = pd.merge(df_OA_lelvel_centroid_distances, df_oa_facility_types,  how='left', left_on=['oa_destination'], right_on = ['oa_name'])

In [ ]:
# convert nan values into 0
df_OA_level_distance_and_facilities = df_OA_level_distance_and_facilities.fillna(0)

In [ ]:
# Remove column oa_name
df_OA_level_distance_and_facilities.drop('oa_name', axis=1, inplace=True)

In [ ]:
# If distance is 0, then update the value to 0.001
df_OA_level_distance_and_facilities.loc[df_OA_level_distance_and_facilities['distance_miles'] == 0, 'distance_miles'] = 0.001

In [ ]:
df_school_destination_dataframe_capacity

In [ ]:
##################################

In [ ]:
spenser_activity_location_path = r'' # use your path

spenser_agents_no_processed_path = r''  # use your path

In [ ]:
# Spilts persons Dataframe into MSOA areas (whole households are stored in OA areas)
grps_persons_msoa_split = df_school_destination_dataframe_capacity.groupby('Area_MSOA')



In [ ]:
len(grps_persons_msoa_split)

In [ ]:
msoa_num = 0
msoa_tot = df_school_destination_dataframe_capacity.Area_MSOA.nunique()

In [ ]:
t0 = time.time()

agents_processsed = 0


for msoa_name, persons_grp in grps_persons_msoa_split:

    msoa_num += 1

    # loop through the buildings
    for idx_person, person_row in persons_grp.iterrows():



        #Initialise the variable to an empty string
        ## In case the previous to last trip (different to work) is reached in the for loop, then 
        ### the variable 'previous_activity_to_locate' will change the value and will be located from home location
        previous_activity_to_locate = ''


        agents_processsed +=1

        percentage_processed = round(agents_processsed/len(df_school_destination_dataframe_capacity) * 100,2)

        
        t1 = time.time()
        clear_output(wait=True)
        print(f'MSOA num: {msoa_num} of {msoa_tot}, MSOA name: {msoa_name}')
        print(f'Agent num: {agents_processsed} of {len(df_school_destination_dataframe_capacity)}')
        print(f'time running {(t1 - t0)/3600}')
        print(f'Percentage done: {percentage_processed}')


        # Initialise the flag fo false:
        match_previous_activity_from_home = False


        #print(person_row)



        # spenser information
        spenser_id = person_row['PID_AreaMSOA']
        spenser_hh_id = person_row['HID_AreaOA']
        spenser_msoa =  person_row['Area_MSOA']
        spenser_oa =  person_row['Area_OA']
        spenser_age = person_row['Age']

        ## Household information
        spenser_msoa_hh =  person_row['Area_MSOA']
        spenser_oa_hh =  person_row['Area_OA']

        # workplace information
        school_oa =  person_row['School_OA']
        school_msoa =  person_row['School_MSOA']
        school_id = person_row['School_PSU']

        # nts information of the match to get the trips
        nts_id = person_row['IndividualID']
        nts_day_id = person_row['DayID']
        
        
        ## Identify his/her trips
        df_trips_person =  df_NTS_trips.loc[(df_NTS_trips['IndividualID'] == nts_id) & 
                                           (df_NTS_trips['DayID'] == nts_day_id)]


        trip_purposes_list = df_trips_person['TripPurpTo_B01ID'].unique().tolist()


        # if the person has any trip = escort_work, escort_leisure or escort_other, then save the person id in a dataframe and continue.
        ## this person will be processed at the end, once his/her family members in the household have been allocated with locations to their activities
        ### Then, this person will be assigned the same location as a member of the household with the same purpose (but not escorting)
        if (('escort_work' in trip_purposes_list) or ('escort_leisure' in trip_purposes_list) or ('escort_other' in trip_purposes_list)):

            #Append into the list the id of the SPENSER agent
            id_value = spenser_id

            # save his/her id in a dataframe
            df_spenser_no_processed = pd.DataFrame([id_value], columns=['PID_AreaMSOA'])

            # save the data in a csv file
            df_spenser_no_processed.to_csv(spenser_agents_no_processed_path, encoding='utf-8', mode='a', index=False, header=False)

            continue

        else:


            # sort by 'JourSeq'
            df_trips_person = df_trips_person.sort_values(by=['JourSeq'], ascending=True)

            #print(df_trips_person)
            #sys.exit()

            # count the total number of activities
            number_of_trips = len(df_trips_person)

            # count the activity to be processed (the order)
            counter_trips = 0


            # loop through each of the trips
            for idx_trip, trip_row in df_trips_person.iterrows():
                
                temp_list = []

                trip_purpose = trip_row['TripPurpTo_B01ID']
                trip_distance = trip_row['TripDisIncSW']
                trip_id = trip_row['TripID']
                trip_transport_mode = trip_row['MainMode_B04ID']
                trip_start = trip_row['TripStart']
                trip_end = trip_row['TripEnd']


                df_facilities_selected = pd.DataFrame()
                
                                                
                list_a = ['work', 'escort_home', 'home']

                # if the trip is the previous to last (the one just before going home)
                ## then find the facility from the household
                if ((number_of_trips - counter_trips == 1) and (trip_purpose not in list_a)):

                    # Make the flag possitive and continue to the next trip
                    match_previous_activity_from_home = True

                    # Save the trip purpose in a new variable (previous_activity_to_locate)
                    previous_activity_to_locate = trip_purpose
                    # Save the trip_id in a new variable 
                    previous_activity_trip_id = trip_id
                    # Save the trip_id in a new variable 
                    previous_activity_transport_mode = trip_transport_mode
                    # Save the trip_start in a new variable 
                    previous_activity_start = trip_start
                    # Save the trip_rmd in a new variable 
                    previous_activity_end = trip_end

                    # Get the values of the OA and MSOA levels where the house is located
                    facility_msoa_selected =  spenser_msoa_hh
                    facility_oa_selected =  spenser_oa_hh


                    ## update the variables with the values of the place where next activity will start (home):
                    spenser_msoa =  facility_msoa_selected
                    spenser_oa =  facility_oa_selected


                    # Go to the next trip
                    continue


                ###############################################################################
                
                if ((trip_purpose == 'home' and match_previous_activity_from_home == True) or (trip_purpose == 'medical' or trip_purpose == 'other' or trip_purpose == 'eat' or trip_purpose == 'food_shop' or trip_purpose == 'shop' or trip_purpose == 'leisure_act' or trip_purpose == 'leisure_sport')):
                #if ((trip_purpose == 'medical' or trip_purpose == 'other' or trip_purpose == 'eat'  or trip_purpose == 'shop' or trip_purpose == 'leisure_act' or trip_purpose == 'leisure_sport')):


                    # if last trip before going home is ging to be selected from home (unless previous trip is for work)
                    if (trip_purpose == 'home' and match_previous_activity_from_home == True):


                        # Rename the variable value for supermarkets 
                        ## (facilities are called supermarkets, while NTS purpose is food_shop)
                        if (previous_activity_to_locate == 'food_shop'):

                            previous_activity_to_locate = 'supermarket'



                        temp_list.append(spenser_id)
                        temp_list.append(previous_activity_trip_id)
                        temp_list.append(previous_activity_to_locate)
                        temp_list.append(previous_activity_transport_mode)
                        temp_list.append(previous_activity_start)
                        temp_list.append(previous_activity_end)


                        # Create variables with the column names
                        activity_name = f"{previous_activity_to_locate}_facilities"
                        activity_area = f"{previous_activity_to_locate}_area"
                        activity_amenities = f"{previous_activity_to_locate}_amenities"

                    
                    ####################################################################################

                    # if trip is for medical, other, eating, food_shop, shop, leisure or leisure_sport:
                    ## facility selected based on number of amenities within 150m, area, distance from home and distance from previous activity
                    if (trip_purpose == 'medical' or trip_purpose == 'other' or trip_purpose == 'eat'  or trip_purpose == 'shop' or trip_purpose == 'food_shop' or trip_purpose == 'leisure_act' or trip_purpose == 'leisure_sport'):

                        # Rename the variable value for supermarkets 
                        ## (facilities are called supermarkets, while NTS purpose is food_shop)
                        if (trip_purpose == 'food_shop'):

                            trip_purpose = 'supermarket'


                        temp_list.append(spenser_id)
                        temp_list.append(trip_id)
                        temp_list.append(trip_purpose)
                        temp_list.append(trip_transport_mode)
                        temp_list.append(trip_start)
                        temp_list.append(trip_end)

                        # Create variables with the column names
                        activity_name = f"{trip_purpose}_facilities"
                        activity_area = f"{trip_purpose}_area"
                        activity_amenities = f"{trip_purpose}_amenities"


                    ####################################################################################

        
                    #If trip distance is below 8 miles, used the OA distances 
                    if (trip_distance < 8):


                        # Find the OA levels (centroids) that are at the selected or lower distance from the OA level (centroid) where the agent is located and with at least one facility of the type selected:
                        df_OA_level_selected = df_OA_level_distance_and_facilities.loc[(df_OA_level_distance_and_facilities['oa_origin'] == spenser_oa) & (df_OA_level_distance_and_facilities['distance_miles'] <= trip_distance) & (df_OA_level_distance_and_facilities[activity_name] > 0)]



                        # If there is not any facility found, then filter again within 2 extra miles
                        if ((len(df_OA_level_selected) == 0) and (trip_distance < 5)):

                            trip_distance = trip_distance + 2

                            # Find the OA levels (centroids) that are at the selected or lower distance (original distance + 1mile) from the OA level (centroid) where the agent is located and with at least one facility of the type selected:
                            df_OA_level_selected = df_OA_level_distance_and_facilities.loc[(df_OA_level_distance_and_facilities['oa_origin'] == spenser_oa) & (df_OA_level_distance_and_facilities['distance_miles'] <= trip_distance) & (df_OA_level_distance_and_facilities[activity_name] > 0)]



                        if (len(df_OA_level_selected) > 0):


                            ## Keep only the relevant columns
                            df_OA_level_selected = df_OA_level_selected[['oa_origin', 'oa_destination', 'distance_miles', activity_name, activity_area, activity_amenities]]


                            # Get the unique values of the destination OA levels
                            OA_level_destination_list = df_OA_level_selected['oa_destination'].unique().tolist()

                            #print('OA destination list')
                            #print(len(OA_level_destination_list))
                            #print((OA_level_destination_list))


                            # Get the oa_orign levels that below to the house and the oa_destination that belogn to the activity to take place
                            df_home_activity_dist  = df_OA_lelvel_centroid_distances.loc[(df_OA_lelvel_centroid_distances['oa_origin'] == spenser_oa_hh) & (df_OA_lelvel_centroid_distances['oa_destination'].isin(OA_level_destination_list))]


                            # merge 'df_OA_level_selected' with 'df_OA_lelvel_centroid_distances'
                            df_OA_level_selected = pd.merge(df_OA_level_selected, df_home_activity_dist,  how='left', left_on=['oa_destination'], right_on = ['oa_destination'])


                            # Remove unnecessary colum
                            df_OA_level_selected.drop('oa_origin_y', axis=1, inplace=True)

                            # Rename column
                            df_OA_level_selected.rename(columns={'oa_origin_x': 'oa_origin','distance_miles_x': 'distance_to_facility','distance_miles_y': 'distance_home'}, inplace=True)


                            activity_name_sum = df_OA_level_selected[activity_name].sum()
                            activity_area_sum = df_OA_level_selected[activity_area].sum()
                            activity_amenities_sum = df_OA_level_selected[activity_amenities].sum()


                            df_OA_level_selected['activity_name_norm'] = df_OA_level_selected[activity_name]/activity_name_sum
                            df_OA_level_selected['floor_area_norm'] = df_OA_level_selected[activity_area]/activity_area_sum



                            if (trip_purpose == 'medical' or previous_activity_to_locate == 'medical'):

                                ## selector_factor = number of facilities * area of facilities * (1/dist to home)
                                df_OA_level_selected['selector_factor'] = df_OA_level_selected['activity_name_norm'] * df_OA_level_selected['floor_area_norm'] * (1/df_OA_level_selected['distance_home'])


                            else:


                                df_OA_level_selected['num_ameni_norm'] = df_OA_level_selected[activity_amenities]/activity_amenities_sum


                                ## selector_factor = number of facilities * area of facilities * number of amenities * (1/dist to facility) * (1/dist to home)
                                df_OA_level_selected['selector_factor'] = df_OA_level_selected['activity_name_norm'] * df_OA_level_selected['floor_area_norm'] * df_OA_level_selected['num_ameni_norm'] * (1/df_OA_level_selected['distance_to_facility']) * (1/df_OA_level_selected['distance_home'])


                            if (len(df_OA_level_selected) > 1):


                                # Get the sum of the selector_factor values
                                selector_factor_sum = df_OA_level_selected['selector_factor'].sum()

                                #print('selector factor sum')
                                #print(selector_factor_sum)


                                # if selector_factor_sum = 0, then choose one of them randomly
                                if (selector_factor_sum == 0):

                                    # Select one based on probability value of column 'selector_factor_norm'
                                    df_OA_level_selected = df_OA_level_selected.sample(1)

                                else:   

                                    df_OA_level_selected['selector_factor_norm'] = df_OA_level_selected['selector_factor'] / selector_factor_sum

                                    # Select one based on probability value of column 'selector_factor_norm'
                                    df_OA_level_selected = df_OA_level_selected.sample(1,weights="selector_factor_norm")


                            oa_chosen = df_OA_level_selected['oa_destination'].values[0]


                            # Check if the facilities are identified from the previous OA location (match_previous_activity_from_home = False)
                            ## or from home (match_previous_activity_from_homerevious_activity_from_home = True)
                            if (match_previous_activity_from_home == False):

                                # Select those facilities in the oa selected:
                                df_facilities_selected = df_NE_Facilities.loc[(df_NE_Facilities['oa11cd'] == oa_chosen) & (df_NE_Facilities['activities'].str.contains(trip_purpose))]

                            else:
                                # Select those facilities in the oa selected:
                                df_facilities_selected = df_NE_Facilities.loc[(df_NE_Facilities['oa11cd'] == oa_chosen) & (df_NE_Facilities['activities'].str.contains(previous_activity_to_locate))]


                            #######################################################################################################


                    # If distance is greater (or equal) to 8 miles OR zero facilities were found in the OA levels, then use MSOA levels
                    if (trip_distance >= 8 or (len(df_facilities_selected) == 0)):


                        while(len(df_facilities_selected) == 0):


                            # Find the MSOA levels (centroids) that are at the selected or lower distance from the MSOA level (centroid) where the agent is located and with at least one facility of the type selected:
                            df_MSOA_level_selected = df_MSOA_level_distance_and_facilities.loc[(df_MSOA_level_distance_and_facilities['msoa_origin'] == spenser_msoa) & (df_MSOA_level_distance_and_facilities['distance_miles'] <= trip_distance) & (df_MSOA_level_distance_and_facilities[activity_name] > 0)]


                            # If there is not any facility found, then filter again within 5 extra miles
                            if (len(df_MSOA_level_selected) == 0):

                                trip_distance = trip_distance + 5

                                # Find the MSOA levels (centroids) that are at the selected or lower distance (original distance + 5 miles) from the MSOA level (centroid) where the agent is located and with at least one facility of the type selected:
                                df_MSOA_level_selected = df_MSOA_level_distance_and_facilities.loc[(df_MSOA_level_distance_and_facilities['msoa_origin'] == spenser_msoa) & (df_MSOA_level_distance_and_facilities['distance_miles'] <= trip_distance) & (df_MSOA_level_distance_and_facilities[activity_name] > 0)]


                            if (len(df_MSOA_level_selected) > 0):


                                ## Keep only the relevant columns
                                df_MSOA_level_selected = df_MSOA_level_selected[['msoa_origin', 'msoa_destination', 'distance_miles', activity_name, activity_area, activity_amenities]]


                                # Get the unique values of the destination MSOA levels
                                MSOA_level_destination_list = df_MSOA_level_selected['msoa_destination'].unique().tolist()


                                # Get the msoa_orign levels that below to the house and the msoa_destination that belogn to the activity to take place
                                df_home_activity_dist  = df_MSOA_lelvel_centroid_distances.loc[(df_MSOA_lelvel_centroid_distances['msoa_origin'] == spenser_msoa_hh) & (df_MSOA_lelvel_centroid_distances['msoa_destination'].isin(MSOA_level_destination_list))]


                                # merge 'df_MSOA_level_selected' with 'df_MSOA_lelvel_centroid_distances'
                                df_MSOA_level_selected = pd.merge(df_MSOA_level_selected, df_home_activity_dist,  how='left', left_on=['msoa_destination'], right_on = ['msoa_destination'])


                                # Remove unnecessary colum
                                df_MSOA_level_selected.drop('msoa_origin_y', axis=1, inplace=True)

                                # Rename column
                                df_MSOA_level_selected.rename(columns={'msoa_origin_x': 'msoa_origin','distance_miles_x': 'distance_to_facility','distance_miles_y': 'distance_home'}, inplace=True)


                                # normalise the columns
                                activity_name_sum = df_MSOA_level_selected[activity_name].sum()
                                activity_area_sum = df_MSOA_level_selected[activity_area].sum()
                                activity_amenities_sum = df_MSOA_level_selected[activity_amenities].sum()


                                df_MSOA_level_selected['activity_name_norm'] = df_MSOA_level_selected[activity_name]/activity_name_sum
                                df_MSOA_level_selected['floor_area_norm'] = df_MSOA_level_selected[activity_area]/activity_area_sum


                                if (trip_purpose == 'medical' or previous_activity_to_locate == 'medical'):

                                    ## selector_factor = number of facilities * area of facilities * (1/dist to home)
                                    df_MSOA_level_selected['selector_factor'] = df_MSOA_level_selected['activity_name_norm'] * df_MSOA_level_selected['floor_area_norm'] * (1/df_MSOA_level_selected['distance_home'])


                                else:


                                    df_MSOA_level_selected['num_ameni_norm'] = df_MSOA_level_selected[activity_amenities]/activity_amenities_sum

                                    ## selector_factor = number of facilities * area of facilities * number of amenities * (1/dist to facility) * (1/dist to home)
                                    df_MSOA_level_selected['selector_factor'] = df_MSOA_level_selected['activity_name_norm'] * df_MSOA_level_selected['floor_area_norm'] * df_MSOA_level_selected['num_ameni_norm'] * (1/df_MSOA_level_selected['distance_to_facility']) * (1/df_MSOA_level_selected['distance_home'])


                            if (len(df_MSOA_level_selected) >= 1):


                                # Get the sum of the selector_factor values
                                selector_factor_sum = df_MSOA_level_selected['selector_factor'].sum()


                                # if selector_factor_sum = 0, then choose one of them randomly
                                if (selector_factor_sum == 0):

                                    # Select one based on probability value of column 'selector_factor_norm'
                                    df_MSOA_level_selected = df_MSOA_level_selected.sample(1)

                                else:   

                                    df_MSOA_level_selected['selector_factor_norm'] = df_MSOA_level_selected['selector_factor'] / selector_factor_sum


                                    # Select one based on probability value of column 'selector_factor_norm'
                                    df_MSOA_level_selected = df_MSOA_level_selected.sample(1,weights="selector_factor_norm")


                                msoa_chosen = df_MSOA_level_selected['msoa_destination'].values[0]



                                # Check if the facilities are identified from the previous OA location (match_previous_activity_from_home = False)
                                ## or from home (match_previous_activity_from_homerevious_activity_from_home = True)
                                if (match_previous_activity_from_home == False):


                                    # Select those facilities in the msoa selected:
                                    df_facilities_selected = df_NE_Facilities.loc[(df_NE_Facilities['msoa11cd'] == msoa_chosen) & (df_NE_Facilities['activities'].str.contains(trip_purpose))]

                                else:

                                    # Select those facilities in the msoa selected:
                                    df_facilities_selected = df_NE_Facilities.loc[(df_NE_Facilities['msoa11cd'] == msoa_chosen) & (df_NE_Facilities['activities'].str.contains(previous_activity_to_locate))]


                    #######################################################################################################
                    #######################################################################################################


                    # SELECT THE FACILITY where the agent will go.


                    if (len(df_facilities_selected) > 1):


                        facility_area_sum = df_facilities_selected['floor_area'].sum()
                        facility_amenities_sum = df_facilities_selected['num_ameni'].sum()


                        df_facilities_selected['floor_area_norm'] = df_facilities_selected['floor_area']/facility_area_sum
                        df_facilities_selected['num_ameni_norm'] = df_facilities_selected['num_ameni']/facility_amenities_sum


                        ## selector_factor = number of facilities * area of facilities * number of amenities * (1/dist to facility) * (1/dist to home)
                        df_facilities_selected['selector_factor'] = df_facilities_selected['floor_area_norm'] * df_facilities_selected['num_ameni_norm'] 


                        # Get the sum of the selector_factor values
                        selector_factor_sum = df_facilities_selected['selector_factor'].sum()


                        # if selector_factor_sum = 0, then choose one of them randomly
                        if (selector_factor_sum == 0):

                            # Select one based on probability value of column 'selector_factor_norm'
                            df_unique_facility_selected = df_facilities_selected.sample(1)


                        else:

                            df_facilities_selected['selector_factor_norm'] = df_facilities_selected['selector_factor'] / selector_factor_sum

                            # Select one based on probability value of column 'selector_factor_norm'
                            df_unique_facility_selected = df_facilities_selected.sample(1,weights="selector_factor_norm")


                    else:

                        df_unique_facility_selected = df_facilities_selected


                    # Facility ID value
                    facility_id_chosen = df_unique_facility_selected['id'].values[0]

                    facility_oa_selected = df_unique_facility_selected['oa11cd'].values[0]
                    facility_msoa_selected = df_unique_facility_selected['msoa11cd'].values[0]


                    #######################################################################################################
                    #######################################################################################################

                    # SAVE THE DATA IN A CSV FILE

                    if (facility_id_chosen != ''):

                        # Append in to the list the ID of the facility
                        temp_list.append(facility_id_chosen)

                    else:

                        # Append in to the list the ID of the facility
                        temp_list.append('facility_not_found_due_to_distance')


                    # Create an empty dataframe
                    df_spenser_activity_locations = pd.DataFrame([temp_list], columns=['PID_AreaMSOA', 'DayID','Activity_type', 'Transport_mode', 'Start', 'End', 'facility_ID'])

                    # save the data in a csv file
                    df_spenser_activity_locations.to_csv(spenser_activity_location_path, encoding='utf-8', mode='a', index=False, header=False)



                    ## update the variables with the values of the place where next activity will start:
                    spenser_msoa =  facility_msoa_selected
                    spenser_oa =  facility_oa_selected


                    # Deactivate the flag in case it was activated:
                    ## It is only activated when the previous to last activity is NOT work.
                    ## In this case, the previous to last activity is found from home, not from the previous activity.
                    match_previous_activity_from_home = False


                    # Initialise the facility_id_chosen to an empty string
                    ## This is to avoid selecting the previous location in case there was not found any location for the trip in the loop
                    facility_id_chosen =''

                    # empty dataframe
                    df_unique_facility_selected = pd.DataFrame()

                ###############################################################################   

                # if trip is for home
                if (trip_purpose == 'home' or trip_purpose == 'escort_home'):

                    temp_list = []

                    temp_list.append(spenser_id)
                    temp_list.append(trip_id)
                    temp_list.append(trip_purpose)
                    temp_list.append(trip_transport_mode)
                    temp_list.append(trip_start)
                    temp_list.append(trip_end)
                    temp_list.append(spenser_hh_id)

                    # Get the values of the OA and MSOA levels where the hosue is located
                    facility_msoa_selected =  spenser_msoa_hh
                    facility_oa_selected =  spenser_oa_hh


                    # Create an empty dataframe
                    df_spenser_activity_locations = pd.DataFrame([temp_list], columns=['PID_AreaMSOA', 'DayID','Activity_type', 'Transport_mode', 'Start', 'End', 'facility_ID'])

                    # save the data in a csv file
                    df_spenser_activity_locations.to_csv(spenser_activity_location_path, encoding='utf-8', mode='a', index=False, header=False)



                    ## update the variables with the values of the place where next activity will start:
                    spenser_msoa =  facility_msoa_selected
                    spenser_oa =  facility_oa_selected


                ###############################################################################    
                
                
                # if trip is for work:
                if (trip_purpose == 'education'):


                    temp_list.append(spenser_id)
                    temp_list.append(trip_id)
                    temp_list.append(trip_purpose)
                    temp_list.append(trip_transport_mode)
                    temp_list.append(trip_start)
                    temp_list.append(trip_end)
                    temp_list.append(school_id)


                    # Get the values of the OA and MSOA levels where the school is located
                    facility_msoa_selected = school_msoa
                    facility_oa_selected  = school_oa



                    # Create an empty dataframe
                    df_spenser_activity_locations = pd.DataFrame([temp_list], columns=['PID_AreaMSOA', 'DayID','Activity_type', 'Transport_mode', 'Start', 'End', 'facility_ID'])

                    # save the data in a csv file
                    df_spenser_activity_locations.to_csv(spenser_activity_location_path, encoding='utf-8', mode='a', index=False, header=False)



                    ## update the variables with the values of the place where next activity will start:
                    spenser_msoa =  facility_msoa_selected
                    spenser_oa =  facility_oa_selected




print('Code has finished. Check the results, amigo!')